**ENTRENAMIENTO DE TERSONFLOW PARA DATA DE ELECTROCARDIOGRAMAS**

#NOTACION A USAR *son las variables q no son pertinentes en el estudio
NOTA: Los datos originales poseen una fila que se debe eliminar a mano ya que la misma tiene mas columnas que las otras
1. survival -- the number of months patient survived (has survived, if patient is still alive).  
2. still-alive -- a binary variable.  0=dead at end of survival period, 1 means still alive 
3. age-at-heart-attack -- age in years when heart attack occurred
4. pericardial-effusion -- binary. Pericardial effusion is fluid around the heart.  0=no fluid, 1=fluid
5. fractional-shortening -- a measure of contracility around the heart lower numbers are increasingly abnormal
6. epss -- E-point septal separation, another measure of contractility. Larger numbers are increasingly abnormal.
7. lvdd -- left ventricular end-diastolic dimension.  This is a measure of the size of the heart at end-diastole. Large hearts tend to be sick hearts.
8. wall-motion-score -- a measure of how the segments of the left ventricle are moving
9. wall-motion-index -- equals wall-motion-score divided by number of segments seen.  Usually 12-13 segments are seen in an echocardiogram.  Use this variable INSTEAD of the wall motion score.
10. mult -- a derivate var which can be ignored
11. name -- the name of the patient (I have replaced them with "name")
12. group -- meaningless, ignore it
13. alive-at-1 -- Boolean-valued. Derived from the first two attributes. 0 means patient was either dead after 1 year or had been followed for less than 1 year.  1 means patient was alive at 1 year.

In [1]:
#PAQUETERIAS A USAR 
# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
# Bibliotecas de ayuda
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

**ENTRADA DE LA INFORMACION, PROCESAMIENTO**

In [4]:
# NOTACION DE LAS VARIABLES 
label_total=["survival", "still_alive", "age-at-heart-attack",
       "pericardial-effusion","fractional-shortening", 
       "epss", "lvdd","wall-motion-score","wall-motion-index",
       "mult" , "group", "alive-at-1"];
# ENTRAR LA INFORMACION DESDE UN ARCHIVO INTERNO
data = pd.DataFrame(np.genfromtxt('echocardiogram.data', delimiter=','), columns= label_total)
label_user=["age-at-heart-attack", "pericardial-effusion","fractional-shortening", "epss",
            "lvdd","wall-motion-index", "alive-at-1"];

**PROCESO DE FILTRADO**

In [5]:
## FILTRAR LAS VARIABLES QUE SE UTILIZARAN EN EL ENTRENAMIENTO DE ML
select_data=data[label_user];#DATOS ORIGINALES SIN FILTRAR, COLUMNAS DE INTERES
#ARREGLAR LA COLUMNA DE alive-at-1"];
alive= np.zeros(len(select_data));
for j in range(len(alive)):
    if   data[label_total[0]][j] > 12:#MAS DE 12 MESES
         select_data["alive-at-1"].values[j] = 1; #CUMPLE EL REQUISITO
    elif data[label_total[0]][j] == 12 and data[label_total[1]][j] == 1:#12 MESES Y VIVE
         select_data["alive-at-1"].values[j] = 1; #CUMPLE EL REQUISITO
    elif data[label_total[0]][j] < 12 and data[label_total[1]][j] == 1:# MENOS DE 12 MESES Y VIVE
         select_data["alive-at-1"].values[j] = -1; #NO CUMPLE EL REQUISITO
##MATRIX LOGICA DE DATOS ELIMINAR LO DESCONOCIDO
for i in range(len(label_user)):
    if i==0:
        log=select_data[label_user[i]] !=-1;#print(i);
    else:
        log= np.logical_and(select_data[label_user[i]] !=-1,log)
select_data=select_data[log]; # filtrar los desconocidos           

##VALORES FINALES QUE SE UTILIZARAN PARA ENTRENARSE CORRECTAMENTE
input_data=select_data[{"age-at-heart-attack", "pericardial-effusion","fractional-shortening", "epss", "lvdd","wall-motion-index"}];
output_data=select_data[{"alive-at-1"}];

**PARAMETROS INICIALES DE ENTRENAMIENTO**

In [9]:
n_input = np.int64(len(label_user)-1)   # input layer
n_hidden = np.int64(np.round(np.sqrt(len(label_user)))) # 1st hidden layer
n_output = np.int64(1)   # output 

learning_rate = 1e-4;   n_iterations = 100
batch_size = 3;         dropout = 0.5

X = tf.compat.v1.placeholder("float", [None, n_input]);
Y = tf.compat.v1.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32) 

weights = {
    'w1': tf.Variable(tf.random.normal([n_input, n_hidden])),
    'out': tf.Variable(tf.random.normal([n_hidden, n_output]))
    }

biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
#layer_drop = tf.nn.dropout(layer_1, keep_prob)
output_layer = tf.matmul(layer_1, weights['out']) + biases['out']

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output_layer))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

**INICIALIZAR EL ENTRENAMIENTO**

In [23]:
init = tf.compat.v1.global_variables_initializer()
sess = tf.compat.v1.Session()
sess.run(init)

cost_summary = tf.compat.v1.summary.scalar("Cost", cross_entropy)
acc_summary = tf.compat.v1.summary.scalar("Accuracy", accuracy)
all_summary = tf.compat.v1.summary.merge_all()
writer = tf.summary.FileWriter("Tensorboard", sess.graph)

**ENTRENAMIENTO**

In [25]:
with tf.compat.v1.Session() as sess:
    init = tf.compat.v1.global_variables_initializer()
    sess.run(init)

    for i in range(n_iterations):
        sess.run(train_step, feed_dict={X: input_data.values, Y: output_data.values})
        #if i % 10 == 0:
        summary_results, loss, acc = sess.run([all_summary, cross_entropy, accuracy], feed_dict={X: input_data.values, Y: output_data.values})
        print(f"Iteration {i} \t| Loss = {loss:7.5f} \t| Accuracy = {acc:7.5f}")
        #    writer.add_summary(summary_results, i)
    test_accuracy = sess.run(accuracy, feed_dict={X: input_data.values, Y: output_data.values})
    print("\nAccuracy on test set:", test_accuracy)

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,1]
	 [[node Placeholder_1 (defined at <ipython-input-7-b852f98bfc24>:9) ]]

Original stack trace for 'Placeholder_1':
  File "C:\Users\Dell\.conda\envs\ML\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Dell\.conda\envs\ML\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Dell\.conda\envs\ML\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Dell\.conda\envs\ML\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Dell\.conda\envs\ML\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "C:\Users\Dell\.conda\envs\ML\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Dell\.conda